In [30]:
import os
import pandas as pd
import openai
from gpt_score.gpt3_score import gpt3score
from tqdm import tqdm
import random
import logging
import json
import re

tqdm.pandas()

proxy = 'http://dell-1.star:7890' # 3090 docker
os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

openai.api_key_path = ".openai-key2"

# baidu_dict = json.load(open(".env_baidu", "r"))
baidu_dict = json.load(open(".env_baidu2", "r"))

In [5]:
dev_sql = pd.read_json("data/sql/dev_sql.json")
dev_sql.to_json("data/sql/dev_sql.json", orient="records", force_ascii=False, indent=4)
train_sql = pd.read_json("data/sql/sql_finetune_data.json")
train_sql.to_json("data/sql/sql_finetune_data.json", orient="records", force_ascii=False, indent=4)

In [28]:
import http.client
import hashlib
import urllib
import random
import json

def translate(query, appid, secret_key, 
            from_lang="en", to_lang="zh",
            domain="it",
            url="/api/trans/vip/fieldtranslate"):
    
    salt = random.randint(32768, 65536)
    if domain is not None:
        sign = appid + query + str(salt) + domain + secret_key
    else:
        sign = appid + query + str(salt) + secret_key
    sign = hashlib.md5(sign.encode()).hexdigest()
    myurl = url + "?appid=" + appid + "&q=" + urllib.parse.quote(query) \
            + "&from=" + from_lang + "&to=" + to_lang + "&salt=" \
            + str(salt) + "&sign=" + sign
    if domain is not None:
        myurl += "&domain=" + domain 
    try:
        http_client = http.client.HTTPConnection('fanyi-api.baidu.com')
        http_client.request('GET', myurl)
        response = http_client.getresponse()
        response_str = response.read().decode('utf-8')
        json_data = json.loads(response_str)
        print(json_data)
        res = json_data["trans_result"][0]["dst"]
    except Exception as e:
        print(e)
        res = ""
    finally:
        if http_client:
            http_client.close()
        return res

In [29]:
str_en = "What is the feature type name of feature AirCon?"
str_zh = translate(str_en, baidu_dict["appid"], baidu_dict["secret_key"], domain=None)

{'error_code': '54004', 'error_msg': 'Please recharge'}
'trans_result'


# Batch translation

In [20]:
def translate_wrapper(query):
    return translate(query, baidu_dict["appid"], baidu_dict["secret_key"])

In [22]:
# filename = "data/sql/dev_sql.json"
filename = "data/sql/sql_finetune_data.json"
df = pd.read_json(filename)
df["input_zh"] = df["input"].progress_apply(translate_wrapper)
df.to_json(filename.replace(".json", "_zh.json"), orient="records", force_ascii=False, indent=4)

 18%|█▊        | 1532/8659 [08:12<12:16:27,  6.20s/it]

[Errno -3] Temporary failure in name resolution


 71%|███████   | 6131/8659 [37:13<10:00,  4.21it/s]   

'trans_result'
'trans_result'


 71%|███████   | 6133/8659 [37:13<07:20,  5.74it/s]

...

'trans_result'
'trans_result'


100%|██████████| 8659/8659 [47:00<00:00,  8.48it/s]

'trans_result'
'trans_result'


100%|██████████| 8659/8659 [47:00<00:00,  3.07it/s]


'trans_result'


In [31]:
for idx, row in df.iterrows():
    if row["input_zh"] == "":
        input_zh = translate(row["input"], baidu_dict["appid"], baidu_dict["secret_key"])
        df.loc[idx, "input_zh"] = input_zh
        print(input_zh)

df.to_json(filename.replace(".json", "_zh.json"), orient="records", force_ascii=False, indent=4)

{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'Which customers have the substring "Diana" in their names? Return the customer details.', 'dst': '哪些客户的名字中有子串“Diana”？返回客户详细信息。'}]}
哪些客户的名字中有子串“Diana”？返回客户详细信息。
{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'Find the name of the customer who made an order most recently.', 'dst': '查找最近下订单的客户的姓名。'}]}
查找最近下订单的客户的姓名。
{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'Which product has been ordered most number of times?', 'dst': '哪种产品的订购次数最多？'}]}
哪种产品的订购次数最多？
{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'What is the most frequently ordered product? Tell me the detail of the product', 'dst': '最常订购的产品是什么？告诉我产品的细节'}]}
最常订购的产品是什么？告诉我产品的细节
{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'Find the name and ID of the product whose total order quantity is the largest.', 'dst': '查找订单总量最大的产品的名称和ID。'}]}
查找订单总量最大的产品的名称和ID。
{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'What are the name and ID of the product bought the most.',